# Job title embeddings
In this notebook, we will implement word embeddings for job title using [Word2vec](https://en.wikipedia.org/wiki/Word2vec). By implementing this, you will learn about embedding job titles. this will be useful when calculating similarity for job matching.

Table of contents

- [Word embeddings, word2vec](#Word-embeddings,-word2vec)
- [Job posting data](#Job-posting-data)
- [Reading](#Reading)


## Word embeddings, word2vec
Word2vec is a machine learning model used for learning vector representation of words, called "word embeddings".
Before starting implemenation, let's look at why we would want to learn work embeddings in the first place.

### Why learn word embeddings?
Natural language processing traditonally convert words to discrete symboles or ids. like `cat`, `dog` ->` 101`, `213`  
These encondings provide no useful information about relationships that may exist between individual symbols. representing words as unique, discrete ids furthermore leads to data sparsity, and requires many data. using vector representations can overcome some of these issues.

### Word2vec
Word2vec is a neural network model which allows you to vectorize words.
If you are interested in how to calculate vectors of words, read some of readings listed below.

<img src="assets/word2vec_diagrams.png" width="700">

Each word will be located as a point in hundreds-dimentional vectorspace (practically 300 - 500 dimentions). a well trained set of word vectors will place similar words close to eh other in that space. for instance, `dogs` and `cats` might cluster in one corner, while `war`, `conflict` and `strife` do in another.
Word2vec can learn many associations other than similarity. for instance, it can gauge relations between words of one language, and map them to another.

<img src="assets/word2vec_translation.png" width="700">

You can also do the following calculations with these vectors.

```
Rome - Italy + Japan = Tokyo
```
```
king - man + woman = queen
```

<img src="assets/countries_capitals.png" width="700">

### How are job title embeddings useful?
One of the reasons why we would want to learn "job title embeddings" is that it will be useful when calculating similar jobs with your current job title for job matching, which is difficult to be done by traditonal search-based system.  
We will see potential applications using the embeddings later in this notebook.

|Target job|Similar jobs|
|---|---|
|Web engineer| Server side eingeer, fullstack engineer, front-end engineer|
|Infra engineer| Devops engineer, System reliability engineer|
|Data scientist| Data analyist, Data engineer, Machine learning engineer|

## Job posting data
We need job titles and the description of them to learn what each title means.  
We will use job posting data crawed from [Green Japan](https://www.green-japan.com/) because that service focuses on web/IT industry and we can expect to learn more detailed meanings in the specific industry.
Let's look at the raw data first.

In [4]:
import pandas as pd

pd.read_csv('./data/job_postings.csv')

,_type,about_company,department,job_highlight,job_title,location,page_title,page_url,reason_for_hiring,requirements,salary,summary,tags
0,dict,ミッション「CLOSING THE DISTANCE」を軸に、人と人、様々な情報・サービスを...,NaN,【仕事の魅力】\n・900名規模のグローバルカンパニーで人事労務のキャリアを積むことができる...,人事労務,福岡県,【LINE】社員がやりがいをもって働ける環境を整える、人事労務,https://www.green-japan.com/job/60202,NaN,【必須の経験・スキル】\n・人事労務業務経験3年以上\n【歓迎する経験・スキル】\n・有期雇...,400万円〜,LINEおよび関連サービスの開発・クリエイティブ・運営・企画営業などあらゆる業務を行っていま...,#企画営業
1,dict,ミッション「CLOSING THE DISTANCE」を軸に、人と人、様々な情報・サービスを...,NaN,【仕事の魅力】\n・勢いのある広告サービスの利用規模拡大に携わることができる\n・ワークライ...,BtoBテレマーケティング担当(広告関連サービス/SV候補),福岡県,【LINE】LINE広告関連サービスのテレマーケティング担当(SV候補),https://www.green-japan.com/job/60664,事業拡大による増員,【必須の経験・スキル】\n・勢いのある広告サービスの利用規模拡大に携わることができる\n・ワ...,350万円〜,LINEおよび関連サービスの開発・クリエイティブ・運営・企画営業などあらゆる業務を行っていま...,#企画営業
2,dict,ミッション「CLOSING THE DISTANCE」を軸に、人と人、様々な情報・サービスを...,NaN,【仕事の魅力】\n・グローバルに展開しているゲームのマーケティングを経験できる\n・プラット...,マーケティング・プロモーション担当(LINEゲーム),福岡県,【LINE】ゲームタイトルのプロモーション企画をおこなうマーケティング担当,https://www.green-japan.com/job/60809,事業拡大による増員,【必須の経験・スキル】\n・プロモーション企画の提案・実施経験\n・広告のバナー制作・配信設...,350万円〜,LINEおよび関連サービスの開発・クリエイティブ・運営・企画営業などあらゆる業務を行っていま...,#企画営業
3,dict,＜会社概要＞\n今年で12期のまだまだ若くエネルギッシュな会社です。\n「いつも心にエンタテ...,テクノロジーソリューション事業部への配属となります。,【得られるスキル】 \n・カスタマーサクセスチームの立ち上げ経験 \n・グローバルで事業をグ...,【急成長イスラエルITベンチャーと働く】カスタマーサクセス職,東京都,急成長イスラエルITベンチャーと働く、カスタマーサクセスを大募集！,https://www.green-japan.com/job/54107,最先端ツールでWEB業界を革新中の弊社がさらなる成長を目指すため、\n一緒にカスタマーサクセ...,【必須スキル】 \n・BtoBセールス実務経験 \n・実務レベルの英語力 \n・Excel/...,300万円〜600万円,■Webマーケティング事業\n■テクノロジーソリューション事業\n\n 今回募集を...,#グローバル化\n#イスラエル\n#一気通貫
4,dict,＜会社概要＞\n今年で12期のまだまだ若くエネルギッシュな会社です。\n「いつも心にエンタテ...,テクノロジーソリューション事業部への配属となります。,【得られるスキル】 \n・潜在的な事業課題を導き出す高いヒアリング力 \n・チームの仕組み化...,【最先端ツールでWEB業界を変える】セールス職,東京都,急成長中！イスラエル・シリコンバレースタートアップと働く営業を募集中！,https://www.green-japan.com/job/54101,最先端ツールでWEB業界を革新中の弊社がさらなる成長を目指すため、\n一緒にセールスを行って...,【必須スキル】 \n・複数の関係者と協業できるコミュニケーション能力 \n・BtoBセールス...,300万円〜600万円,■Webマーケティング事業\n■テクノロジーソリューション事業\n\n ギャプライ...,#グローバル化\n#イスラエル\n#一気通貫
5,dict,弊社では、Webサイトからのお問合せや売上増加を実現するために、\n独自開発したユーザーの購...,WEBマーケティング事業運営チームへの配属となります。,■大手企業・上場企業を始め1000本超のLP制作実績から培われたノウハウを1から学ぶことがで...,WEBマーケティングプランナー/ディレクター,東京都,【WEBディレクター/業界営業経験者】あなたのアイディアで、クライアントのビジネスを大きく飛...,https://www.green-japan.com/job/45013,「なぜか、あの商品が気になる・・・・」\n「あのサービスを使ってみたい！」\nそんな風に自然...,【必須】\n※下記の中で一つ以上、当てはまるご経験をお持ちの方\n・制作会社でのWeb サイ...,350万円〜600万円,■Webマーケティング事業\n■テクノロジーソリューション事業\n\n ■お客様へ...,#グローバル化\n#イスラエル\n#一気通貫
6,dict,オープンポジションです。成長企業を支える管理部の中でメンバーと一緒にバックオフィス業務をお願...,管理部または経営企画室への配属を予定しています。,NaN,オープンポジション（バックオフィス系）,東京都,オープンポジション・上場準備会社のバックオフィスで一緒に成長しませんか？,https://www.green-japan.com/job/60651,当社は2018年で創立14期目となりますが、業績急拡大への対応と、数年内の上場準備のために、...,◆必須条件 \n・事業会社でのバックオフィスでの就業経験\n◆求める人物像 \n・粘り強くタ...,400万円〜700万円,■Webマーケティング事業\n■テクノロジーソリューション事業\n\n ＜想定業務...,#グローバル化\n#イスラエル\n#一気通貫
7,dict,NaN,技術部,新しいポジションであるため、問題を発見する力や課題解決へ導く力、 \n自分で意見を発信させて...,インフラエンジニア,東京都,インフラエンジニア(クラウド)AWSやAzureの設計構築にも携われる■自社勤務■フレックス...,https://www.green-japan.com/job/60269,2016年7月より前身の企業から新たな社名「UPWARD」に変更し、新たなスタートを切った当...,★Webアプリケーション基盤としてサーバー設計および運用経験がある方 \n※パブリッククラウ...,400万円〜800万円,～営業社員の強い味方★クラウドサービス「UPWARD」～ \n○社会貢献性の高いサービスに携...,#代理店\n#GIS\n#マルチデバイス\n#社会貢献\n#中央区\n#オープンソース\n#...
8,dict,全国で活躍する営業社員を、 \nあなたの「品質管理」が影から支えます。\n訪問営業を行う皆様...,技術部,新しいポジションであるため、問題を発見する力や課題解決へ導く力、 \n自分で意見を発信させて...,QAマネージャー,東京都,【QAマネージャー候補】自社サービスの品質管理に携わる/年俸550万円～/フレックスタイム制...,https://www.green-japan.com/job/60257,2016年7月より前身の企業から新たな社名「UPWARD」に変更し、新たなスタートを切った当...,★学歴不問 \n★ソフトウェアの品質管理業務を経験、理解している方 \n※マネジメント・それ...,400万円〜800万円,～営業社員の強い味方★クラウドサービス「UPWARD」～ \n○社会貢献性の高いサービスに携...,#代理店\n#GIS\n#マルチデバイス\n#社会貢献\n#中央区\n#オープンソース\n#...
9,dict,弊社は、”かんたんで使いやすい位置情報サービスを提供し、ビジネスマンが快適に働ける世界を実現...,営業部,課題解決力,未来に語り継がれる★カスタマーサクセス★,東京都,★★未来を提案★★ クラウド＋効率化＝UPWARD カスタマーサクセス大募集！,https://www.green-japan.com/job/60092,業務拡大により増員,・B2B分野での営業または顧客分析、営業戦略の立案などの業務経験をお持ちの\n・顧客満足向上...,400万円〜800万円,～営業社員の強い味方★クラウドサービス「UPWARD」～ \n○社会貢献性の高いサービスに携...,#代理店\n#GIS\n#マルチデバイス\n#社会貢献\n#中央区\n#オープンソース\n#...


The dataset has 12130 postings with 13 fields: `job_title`, `summary`, `requirements`, `salary`, `location` etc. some explain job title, other don't. we can decide which fields to use for learning.

## Reading
Here are some resources which you can learn more about word2vec after workshop.
- [Conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
- [Tensorflow tutorial](https://www.tensorflow.org/tutorials/word2vec)
- [Gensim documentation](https://radimrehurek.com/gensim/models/word2vec.html)
- [Udacity material](https://github.com/udacity/deep-learning/blob/master/embeddings/Skip-Gram_word2vec.ipynb)